In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sbn
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import  tqdm, trange

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda')

In [3]:
class MyNet(nn.Module):

    def __init__(self, w_layers, pred_layers):

        super().__init__()
              
        # Weaher variables
        w_layers_list = nn.ModuleList()
        for i, o, k, d in w_layers:
            w_layer = nn.Sequential(
                nn.Conv1d(i, o, k),
                nn.AvgPool1d(2),
                nn.LeakyReLU(),
                nn.BatchNorm1d(o),
                nn.Dropout(d)
            )
            w_layers_list.append(w_layer)
        w_layers_list.append(nn.AdaptiveAvgPool1d(1))
        self.w_layers = nn.Sequential(*w_layers_list)

        
        # Management variables
        pred_layers_list = nn.ModuleList()
        for i, o, d in pred_layers:
            pred_layer = nn.Sequential(
                nn.Linear(i, o),
                nn.LeakyReLU(),
                nn.BatchNorm1d(o),
                nn.Dropout(d)
            )
            pred_layers_list.append(pred_layer)
        pred_layers_list.append(nn.Linear(o, 1))
        self.pred_layers = nn.Sequential(*pred_layers_list)
        
   
    def forward(self, Ws):
        
        feat = self.w_layers(Ws).view(Ws.shape[0], -1)
        pred = self.pred_layers(feat)
        return (torch.tanh(pred))


In [265]:
def transform(w):
    ws = np.array([[[5e4,50,50,5,100.0]]])
    w = w / ws
    w = np.moveaxis(w, 1, 2)
    wd = np.linspace(-0.9,2.1,300)[None,None]
    wd = wd.repeat(len(w), 0)
    w = np.concatenate([w, wd], 1)
    w = torch.tensor(w, dtype=torch.float, device = device)
    return(w)

def back_transform(w):
    w = w[:,:-1].cpu().data.numpy()
    w = np.moveaxis(w, 2, 1)
    ws = np.array([[[5e4,50,50,5,100.0]]])
    w = w * ws
    return(w)


def get_adv(x):
    x_opt = x.clone()
    x_opt.requires_grad = True

    optimizer = torch.optim.Adam([x_opt], lr=0.01)

    for i in trange(100):

        # Limpa os gradientes
        optimizer.zero_grad()

        # Obtem o output
        outputs = model(x_opt)

        # Calcula a perda pela loss function
        loss = -outputs.mean()

        # Use an l2 penalty:
        loss += torch.norm(x - x_opt, 2, dim = 1).mean()

        # Obtem os gradientes
        loss.backward()

        # Atualiza os parâmetros
        optimizer.step()

        # Clip to the valid range of values:
        x_opt.data = torch.clamp(x_opt.data, 0, 1)
        x_opt.data[:,-1] = x_opt[:,-1]

    return(x_opt)

In [258]:
d = 0.0
w_layers =  [[6,12,3,d],[12,15,5,d],[15,20,7,d],[20,25,5,d],[25,100,3,d]]
pred_layers = [[100,50,d],[50,50,d], [50,25,d]]


# model = MyNet(w_layers, pred_layers)
# model = model.to(device)
    
# model_file_name = '../data/model_cnn_rnd_0.1.pth'
# model.load_state_dict(torch.load(model_file_name, map_location=device))
# model.eval()
# print('Model OK')

In [230]:
# Values used to scale the weather data:
ws = np.array([[[5e4,50,50,5,100.0]]])
w_std = np.array([12.4470, 11.6719, 12.3756,  6.6907,  9.4876, 0.0])
w_std = torch.tensor(w_std, device = device, dtype = torch.float)[:,None]

In [231]:
ydf = pd.read_hdf('../data/PSCE_TILE.h5', key = 'SIM').set_index('SIM')
ydf['Yield'] = (ydf.TWSO/2e4)

In [147]:
wdir = '/home/rodrigo7/Apsim_test/MASAGRO/DAYMET_TILE'
pxy = np.stack(np.meshgrid(np.arange(40), np.arange(40)), -1).reshape(-1, 2)
wfiles = [f'{wdir}/DAYMET_9584_{px:02d}_{py:02d}.csv' for px, py in pxy]

In [148]:
# wridx = []
# for wfile in wfiles:   
#     ridx = 10 * np.arange(8,10) + np.random.randint(0, 10, 2)
#     wridx.append(ridx)
# wridx = np.array(wridx)
# np.save('../data/adv_idx.npy', wridx)
wridx = np.load('../data/adv_idx.npy')

In [149]:
w_SIMl = []
for wfile, ridx in zip(tqdm(wfiles), wridx):   
    site = os.path.basename(wfile).replace('.csv', '')

    w = pd.read_csv(wfile, skiprows = 13)
    w.DAY = pd.to_datetime(w.DAY, format = '%Y%m%d').dt.date
    sydf = ydf.loc[site]

    for crop_start_date in sydf.SIM_DATE.values[ridx]:
        cs_date = np.where(w.DAY == crop_start_date)[0][0]
        wrng = slice(cs_date-90, cs_date+210)
        w_SIMl.append(w.iloc[wrng].copy())

w_seed = [w.iloc[:,[1,2,3,4,6]].values for w in w_SIMl]
w_seed = transform(np.stack(w_seedl))   

In [266]:
train_methods = ['none', 'rnd', 'adv']

for REP in range(1, 6):
    for train_method in train_methods:
        for PCT in [1,5]:
            model = MyNet(w_layers, pred_layers)
            model = model.to(device)
            model_file_name = f'../data/model_cnn_{train_method}_{PCT}_{REP}.pth'
            print(model_file_name)
    
            model.load_state_dict(torch.load(model_file_name, map_location=device))
            model.eval()
            
            w_adv = get_adv(w_seed)
            w_adv = back_transform(w_adv)

            for i, (sw_adv, w_SIM) in enumerate(zip(w_adv, w_SIMl)):
                w_SIM = w_SIM.copy()
                w_SIM.iloc[:,[1,2,3,4,6]] = sw_adv
                w_SIM.VAP = np.clip(w_SIM.VAP, 0.06, 199.3)

                plant_date = format(w_SIM.DAY.iloc[90], '%Y%m%d')
                save_file = wfile.replace('.csv', f'_{plant_date}_opt_{train_method}_{PCT}_{REP}.csv')
                with open(save_file, 'w') as sf:
                    with open(wfile) as f:
                        for r in range(14):
                            sf.writelines(f.readline())

                w_SIM.DAY = pd.to_datetime(w_SIM.DAY).dt.strftime('%Y%m%d')
                w_SIM.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)


../data/model_cnn_none_1_1.pth



../data/model_cnn_none_5_1.pth



../data/model_cnn_rnd_1_1.pth



../data/model_cnn_rnd_5_1.pth



../data/model_cnn_adv_1_1.pth


...


../data/model_cnn_adv_1_2.pth



../data/model_cnn_adv_5_2.pth



../data/model_cnn_none_1_3.pth



../data/model_cnn_none_5_3.pth


FileNotFoundError: [Errno 2] No such file or directory: '../data/model_cnn_none_5_3.pth'